# Fit Rankine, Holland, Willoughby and Chavas profiles on SAR data
Note that so far the Rmax is searched in the first 200km around the TC center (in the code this impacts how the functions are initialized, but also when the curves are plot (on the smaller scale graph).

For the fitting, we constrain Rmax > 5km which works well for a lot of TCs. 

In the papers of Holland, Willoughby and Chavas, the wind used is the tangential wind (same for Rankine). 

TODO:

==> Which profiles work when? etc...



In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

from scipy.interpolate import griddata
from scipy.optimize import curve_fit

import glob

import functions as f

In [2]:
### PATHS
# dir_path = '/home/arthur/data/cyclobs/rotated_files/'
dir_path = '/home/arthur/data/cyclobs/rotated_files/clean_dataset/'
all_data = glob.glob(dir_path + "*.nc")
print("Total number of files:", len(all_data))
# Savepath to be modified directly in the function

### PARAMS
PARAMS = {
    'r_window_len':          501,  # Scale for radius (km)
    'rmax_window':           300,  # Window in whick we look for Rmax (e.g [0, 200km])
    'chavas_vfit':           17,
    'rank_hol_will_vmin':    True, # Uses Vmin as a free parameter if True. If False, Vmin = 0
    'chavas_vmin':           False, # Translates the profile from Vmin if True
    
    'r_Rmax_axis':           True, # If True, uses r* = r/Rmax as x-axis
    'v_Vmax_axis':           True, # If True, uses V* = V/Vmax as y-axis
    'r_Rmax_scale':          16.,
    'r_Rmax_num_pts':        321,
    
    'use_curve_fit':         True,
    'tangential_wind_speed': True, # If False, total wind speed is used
    'print_values':          False, 
    
    'save_dir':              "/home/arthur/results/windProfiles/test12/",
    'save_comparison':       True,
    'save_scatter':          False
    }

Total number of files: 322


In [3]:
# INITIALIZE DATA STRUCTURE
INI = {           # Initial values
    'Rankine':    [], # x, alpha, Vmin, Rmax
    'Holland':    [], # Lat, pn, pc, Vmin, Rmax, Vmax
    'Willoughby': [], # n, X1, Vmin, Rmax, Vmax
    'Chavas':     []  # Vmax, Vmin, Rfit, Vfit, fcor, Cdvary, Cd, w_cool, CkCdvary, CkCd, eye_adj, alpha_eye
    } 
FIT = {           # Fit values
    'Rankine':    [], # x, alpha, Vmin, Rmax
    'Holland':    [], # Lat, pn, pc, Vmin, Rmax, Vmax
    'Willoughby': [], # n, X1, Vmin, Rmax, Vmax
    'Chavas':     []  # rr, VV, rmax, r0, rmerge, Vmerge
    }
# GAP WITH OBSERVATIONS
NB_CAT    = [None] * 6
DIFF      = [None] * 6
RMAX_OBS  = [None] * 6
RMAX_FIT  = [None] * 6
VMAX_OBS  = [None] * 6
VMAX_FIT  = [None] * 6
FIT_PARAMS= [None] * 6
r_axis_length = PARAMS['r_window_len']
if PARAMS['r_Rmax_axis']:
    r_axis_length = PARAMS['r_Rmax_num_pts']
for i in range(6):
    # DIFF[0] = Storm
    # DIFF[i] = Cat. i
    DIFF[i] = {
    'Rankine':    np.empty((0,0)),
    'Holland':    np.empty((0,0)),
    'Willoughby': np.empty((0,0)),
    'Chavas':     np.empty((0,0))
    }
    NB_CAT[i] = {
    'Rank-Hol-Will': np.empty((0,0)),
    'Chavas':        np.empty((0,0))
    }
    RMAX_OBS[i] = []
    RMAX_FIT[i] = {
    'Rankine':    [],
    'Holland':    [],
    'Willoughby': [],
    'Chavas':     [] 
    }
    VMAX_OBS[i] = []
    VMAX_FIT[i] = {
    'Rankine':    [],
    'Holland':    [],
    'Willoughby': [],
    'Chavas':     [] 
    }
    FIT_PARAMS[i] = {
    'Rankine':    [],
    'Holland':    [],
    'Willoughby': [],
    'Chavas':     [] 
    }

In [4]:
def calculate_diff_by_cat(cat, Rmax, Vmax, r, spdm, INI, FIT, DIFF, NB_CAT, PARAMS):   
    # Compute fitted profiles
    V_rankine              = f.rankine_profile(r, *FIT['Rankine'])
    V_holland              = f.holland_profile(r, *FIT['Holland'])
    V_willoughby_no_smooth = f.willoughby_profile_no_smooth(r, *FIT['Willoughby'])
    V_chavas               = FIT['Chavas'][1]         # Different from other profiles: here V_chavas has already been computed before, and is stored in FIT['Chavas'][1]
    r_chavas               = FIT['Chavas'][0] / 1000. # Convert from m to km 
    
    # Re-scale Chavas
    lower_bound   = np.argmax(r_chavas >= r[0] - 0.5) # find the index i so that r_chavas[i] = r[0], i.e to translate Cavas if r[0] = 5km for instance
    r_chavas      = r_chavas[lower_bound:]
    V_chavas      = V_chavas[lower_bound:]
    # TODO: use proper interpolation to do what follows
    ind_chavas500 = [np.argwhere(r_chavas >= i)[0] for i in range(0, min(len(r), int(r_chavas[-1])))] # compute the indices of r = 0, 1, 2, ... ==> e.g convert [0.8, 0.9, 0.9, 1.1, 1.2, 1.5, 1.7, 1.9, 2.1, 2.3] to [0, 3, 8]
    ind_chavas500 = [int(i) for i in ind_chavas500]
    V_chavas500   = [V_chavas[i] for i in ind_chavas500]
    
    # Initialize NB_CAT for each profile
    nbcat_rank_hol_will = [1] * len(spdm)
    nbcat_chavas        = [1] * len(V_chavas500)
    nbcat_chavas        = np.concatenate((nbcat_chavas, [0] * (PARAMS['r_window_len'] - len(V_chavas500))), axis=0)
    if len(V_chavas500) < len(spdm):
        V_chavas500 = np.concatenate((V_chavas500, spdm[len(V_chavas500):]), axis=0)
        
    # Compute difference between obs and profile
    diff_rankine = np.subtract(V_rankine, spdm)
    diff_holland = np.subtract(V_holland, spdm)
    diff_willou  = np.subtract(V_willoughby_no_smooth, spdm)
    diff_chavas  = np.subtract(V_chavas500, spdm)
    if PARAMS['r_window_len'] - len(spdm) > 0:
        diff_rankine_filled = np.concatenate((diff_rankine, [0] * (PARAMS['r_window_len'] - len(spdm))), axis=0)
        diff_holland_filled = np.concatenate((diff_holland, [0] * (PARAMS['r_window_len'] - len(spdm))), axis=0)
        diff_willou_filled  = np.concatenate((diff_willou,  [0] * (PARAMS['r_window_len'] - len(spdm))), axis=0)
        diff_chavas_filled  = np.concatenate((diff_chavas,  [0] * (PARAMS['r_window_len'] - len(spdm))), axis=0)
        nbcat_rank_hol_will_filled = np.concatenate((nbcat_rank_hol_will, [0] * (PARAMS['r_window_len'] - len(spdm))), axis=0)
    
    # Normalize by Vmax to obtain V* = V/Vmax
    if PARAMS['v_Vmax_axis']:
        diff_rankine = np.divide(diff_rankine, Vmax)
        diff_holland = np.divide(diff_holland, Vmax)
        diff_willou  = np.divide(diff_willou, Vmax)
        diff_chavas  = np.divide(diff_chavas, Vmax)
        if PARAMS['r_Rmax_axis'] == False and PARAMS['r_window_len'] - len(spdm) > 0:
            diff_rankine_filled = np.divide(diff_rankine_filled, Vmax)
            diff_holland_filled = np.divide(diff_holland_filled, Vmax)
            diff_willou_filled  = np.divide(diff_willou_filled, Vmax)
            diff_chavas_filled  = np.divide(diff_chavas_filled, Vmax)
        
    # Determine the cat. index
    cat = np.array(cat)
    if cat == 'storm' or cat == 'dep':
        i = 0
    else: # then it's 'cat-0', 1, ..., or 5
        i = int(str(cat)[-1])
    
    # Update DIFF and NB_CAT
    r_star = np.linspace(0., PARAMS['r_Rmax_scale'], num=PARAMS['r_Rmax_num_pts']) # axis of reference
    r_Rmax = np.divide(r, Rmax)
    if DIFF[i]['Rankine'].shape == (0, 0): # else we will use np.append()
        DIFF[i]['Rankine']    = np.expand_dims(np.interp(r_star, r_Rmax, diff_rankine), axis=0)
        DIFF[i]['Holland']    = np.expand_dims(np.interp(r_star, r_Rmax, diff_holland), axis=0)
        DIFF[i]['Willoughby'] = np.expand_dims(np.interp(r_star, r_Rmax, diff_willou), axis=0)
        DIFF[i]['Chavas']     = np.expand_dims(np.interp(r_star, r_Rmax, diff_chavas), axis=0)            
    else:
        DIFF[i]['Rankine'] = np.append(DIFF[i]['Rankine'], np.expand_dims(np.interp(r_star, r_Rmax, diff_rankine), axis=0), axis=0) # CAVEAT: if r[500]/Rmax < 16 then np.interp() still fills the vector with the value of diff_rankine[-1]
        DIFF[i]['Holland'] = np.append(DIFF[i]['Holland'], np.expand_dims(np.interp(r_star, r_Rmax, diff_holland), axis=0), axis=0)
        DIFF[i]['Willoughby'] = np.append(DIFF[i]['Willoughby'], np.expand_dims(np.interp(r_star, r_Rmax, diff_willou), axis=0), axis=0)
        DIFF[i]['Chavas'] = np.append(DIFF[i]['Chavas'], np.expand_dims(np.interp(r_star, r_Rmax, diff_chavas), axis=0), axis=0)
    if NB_CAT[i]['Rank-Hol-Will'].shape == (0, 0): # else we will use np.append()
        NB_CAT[i]['Rank-Hol-Will'] = np.expand_dims(np.interp(r_star, r_Rmax, nbcat_rank_hol_will), axis=0)
        NB_CAT[i]['Chavas']        = np.expand_dims(np.interp(r_star, r_Rmax, nbcat_chavas[:len(r_Rmax)]), axis=0)
    else:
        NB_CAT[i]['Rank-Hol-Will'] = np.append(NB_CAT[i]['Rank-Hol-Will'], np.expand_dims(np.interp(r_star, r_Rmax, nbcat_rank_hol_will), axis=0), axis=0)
        NB_CAT[i]['Chavas']        = np.append(NB_CAT[i]['Chavas'], np.expand_dims(np.interp(r_star, r_Rmax, nbcat_chavas[:len(r_Rmax)]), axis=0), axis=0)
       
    return DIFF, NB_CAT

'''
def get_mean_std(x, n):
    mean = np.divide(np.sum(x, axis=0), np.expand_dims(n, axis=0))
    gap  = (x - mean) ** 2
    var  = np.divide(np.sum(gap, axis=0), n)
    std  = np.sqrt(var)
    return mean, std
'''

def get_mean_std(x, n):
    masked = np.ma.masked_where(n == 0, x)
    mean   = np.ma.mean(masked, axis=0)
    std    = np.ma.std(masked, axis=0)
    return mean, std

def plot_comp_by_cat(DIFF, NB_CAT, PARAMS):
    # Initialize radius
    xlabel = 'r (km)'
    ylabel = 'Wind speed mean difference (m/s)'
    if PARAMS['r_Rmax_axis']:
        r      = np.linspace(0., PARAMS['r_Rmax_scale'], num=PARAMS['r_Rmax_num_pts']) # axis of reference
        xlabel = 'r* = r/Rmax'
    else:   
        r      = np.arange(501)
    if PARAMS['v_Vmax_axis']:
        ylabel = 'V* = V/Vmax mean difference'
    
    # Define figure attributes
    filename = f.get_filename('all_profiles_comparison_by_category', PARAMS)
    fig = plt.figure(figsize=(25, 35))
    plt.suptitle("Mean difference between SAR wind speed and common parametric profiles", fontsize=18)
    COLORS = {
        'Willoughby': 'orchid',
        'Chavas':     'forestgreen'
    }
    subtitles = ['Storm', 'Cat.1', 'Cat.2', 'Cat.3', 'Cat.4', 'Cat.5']
    
    # Print TC number in each cat:
    print("Number of TCs in each categories:")
    print("Storm:  ", int(np.max(np.sum(NB_CAT[0]['Chavas'], axis=0))))
    for i in range(1, 6):
        print("Cat.", i, ":", int(np.max(np.sum(NB_CAT[i]['Rank-Hol-Will'], axis=0))))
    
    # Plot curves
    for i in range(6):
        RMSE = {
        'Willoughby': 0.,
        'Chavas':     0. 
        }
        ax = fig.add_subplot(6, 1, i + 1)
        plt.gca().set_title(subtitles[i], fontsize=16)
        for profile in ['Willoughby', 'Chavas']:
            if profile == 'Chavas':
                mean_diff, std_diff = get_mean_std(DIFF[i][profile], NB_CAT[i][profile])
            else:
                mean_diff, std_diff = get_mean_std(DIFF[i][profile], NB_CAT[i]['Rank-Hol-Will'])
            plt.plot(r, mean_diff, color=COLORS[profile], label=profile)
            plt.fill_between(r, mean_diff - std_diff, mean_diff + std_diff,
                            facecolor = COLORS[profile],
                            color     = COLORS[profile],
                            alpha     = 0.2
                            )
            
            # Compute and plot RMSEs
            RMSE[profile] = f.rmse(mean_diff, [0.] * len(mean_diff)) * 100
        plt.text(0.4, 0.78, 'Mean gap (percentage of Vmax): \nWilloughby = {:.2f}   Chavas = {:.2f}'.format(RMSE['Willoughby'], RMSE['Chavas']), fontsize=14, transform = ax.transAxes, bbox=dict(facecolor='white', edgecolor='k', pad=10.0))
        
        # Set legends
        plt.xlabel(xlabel, fontsize=18)
        plt.ylabel(ylabel, fontsize=18)
        ax.set_xticks(np.arange(0, PARAMS['r_Rmax_scale'], 1))
        if PARAMS['v_Vmax_axis']:
                ax.set_ylim(-0.3, 0.8)
        plt.legend();plt.grid()
    
    # Save figure
    if PARAMS['save_comparison']:
        plt.savefig(PARAMS['save_dir'] + filename)
    else:
        plt.show() # f.
        
    return None

In [5]:
### ======================= WITHOUT QUADRANT ======================= 
# FIT AND PRINT ALL THE PROFILES ON ALL THE DATA
i   = 0
for file in all_data[:20]: # Set minimum 20
    i += 1
    print(i, "=>    ", file)
    # Open file and compute mean wind speed
    ds      = xr.open_dataset(file)
    if PARAMS['tangential_wind_speed']:
        spdm = f.compute_mean_tangential_wind_spd(ds, r_window_len=PARAMS['r_window_len']) # TANGENTIAL WIND SPEED
    else:
        spdm = f.compute_mean_wind_spd(ds, r_window_len=PARAMS['r_window_len'])            # TOTAL WIND SPEED
    
    # Debug
    if 1 == 0: # DEBUG
        f.print_ds(ds)
        f.print_spd(ds)
        print(spdm)
    
    # Initialize and fit profile
    r, spdm, first_valid_index = f.initialize_radius(spdm)
    
    INI['Rankine']    = f.initialize_rankine(spdm, x=0.5, alpha=1.,                                PARAMS=PARAMS)
    INI['Holland']    = f.initialize_holland(spdm, Lat=np.float64(ds['lat_ref']), pn=1005, pc=950, PARAMS=PARAMS)
    INI['Willoughby'] = f.initialize_willoughby(spdm, n=1.,                                        PARAMS=PARAMS)
    INI['Chavas']     = f.initialize_chavas(spdm, Lat=np.float64(ds['lat_ref']),                   PARAMS=PARAMS)
    
    FIT['Rankine']    = f.fit_rankine(r, spdm, *INI['Rankine'],                 PARAMS=PARAMS)
    FIT['Holland']    = f.fit_holland(r, spdm, *INI['Holland'],                 PARAMS=PARAMS)
    FIT['Willoughby'] = f.fit_willoughby_no_smooth(r, spdm, *INI['Willoughby'], PARAMS=PARAMS)
    FIT['Chavas']     = f.fit_chavas(*INI['Chavas'],                            PARAMS=PARAMS)
    
    Rmax, Vmax = INI['Holland'][4], INI['Holland'][5]
    
    # Comparison by category, function of r_star
    DIFF, NB_CAT = calculate_diff_by_cat(ds['current_category'], Rmax, Vmax, r, spdm, INI, FIT, DIFF, NB_CAT, PARAMS)
    # Compute mean fitted parameters by category
    FIT_PARAMS   = f.calculate_fitted_params_by_cat(ds['current_category'], FIT, FIT_PARAMS, PARAMS)
    # Scater-plots
    RMAX_OBS, RMAX_FIT, VMAX_OBS, VMAX_FIT = f.add_to_scatter_list(ds['current_category'], r, Rmax, Vmax, FIT, RMAX_OBS, RMAX_FIT, VMAX_OBS, VMAX_FIT, PARAMS)

### Plot the comparison
plot_comp_by_cat(DIFF, NB_CAT, PARAMS)
# f.plot_fitted_params_by_cat(FIT_PARAMS, PARAMS)
# f.plot_scatter_rmax(RMAX_OBS, RMAX_FIT, PARAMS)
# f.plot_scatter_vmax(VMAX_OBS, VMAX_FIT, PARAMS)

1 =>     /home/arthur/data/cyclobs/rotated_files/clean_dataset/s1b-ew-owi-cm-20180902t143708-20180902t143912-000003-01720F_ll_gd_rotated.nc


/home/arthur/scripts/windProfiles/symmetric/functions.py:275: RuntimeWarning: divide by zero encountered in true_divide
  V_ER11 = (1. / rr_ER11) * (Vmax * rmax + .5 * fcor * rmax ** 2) * ((2 * (rr_ER11 / rmax) ** 2) / (2 - CkCd + CkCd * (rr_ER11 / rmax) ** 2)) ** (1 / (2 - CkCd)) - .5 * fcor * rr_ER11
/home/arthur/scripts/windProfiles/symmetric/functions.py:275: RuntimeWarning: invalid value encountered in multiply
  V_ER11 = (1. / rr_ER11) * (Vmax * rmax + .5 * fcor * rmax ** 2) * ((2 * (rr_ER11 / rmax) ** 2) / (2 - CkCd + CkCd * (rr_ER11 / rmax) ** 2)) ** (1 / (2 - CkCd)) - .5 * fcor * rr_ER11
/home/arthur/scripts/windProfiles/symmetric/functions.py:189: RuntimeWarning: invalid value encountered in true_divide
  VV = (Mm / rmax) * (MMfracMm / rrfracrm) - .5 * fcor * rmax * rrfracrm #[ms-1]


2 =>     /home/arthur/data/cyclobs/rotated_files/clean_dataset/rs2--owi-cm-20150601t015945-20150601t020101-00003-BDBF8_ll_gd_rotated.nc
3 =>     /home/arthur/data/cyclobs/rotated_files/clean_dataset/s1a-ew-owi-cm-20160827t092124-20160827t092414-000003-014249_ll_gd_rotated.nc
4 =>     /home/arthur/data/cyclobs/rotated_files/clean_dataset/s1a-ew-owi-cm-20170207t015253-20170207t015558-000003-018D33_ll_gd_rotated.nc
5 =>     /home/arthur/data/cyclobs/rotated_files/clean_dataset/rs2--owi-cm-20150509t232412-20150509t232525-00003-E0BD7_ll_gd_rotated.nc


/home/arthur/scripts/windProfiles/symmetric/functions.py:640: RuntimeWarning: Mean of empty slice
  spdm   = np.nanmean(spd, axis=0)


6 =>     /home/arthur/data/cyclobs/rotated_files/clean_dataset/s1a-ew-owi-cm-20181002t211106-20181002t211310-000003-029DF5_ll_gd_rotated.nc
7 =>     /home/arthur/data/cyclobs/rotated_files/clean_dataset/s1a-iw-owi-cm-20170504t071401-20170504t071505-000003-01B35C_ll_gd_rotated.nc
8 =>     /home/arthur/data/cyclobs/rotated_files/clean_dataset/s1b-ew-owi-cm-20191106t194803-20191106t195007-000003-02377F_ll_gd_rotated.nc
9 =>     /home/arthur/data/cyclobs/rotated_files/clean_dataset/s1a-iw-owi-cm-20170921t224421-20170921t224626-000003-01F204_ll_gd_rotated.nc
10 =>     /home/arthur/data/cyclobs/rotated_files/clean_dataset/s1b-ew-owi-cm-20190428t122321-20190428t122602-000003-01E172_ll_gd_rotated.nc
11 =>     /home/arthur/data/cyclobs/rotated_files/clean_dataset/s1b-ew-owi-cm-20180303t150001-20180303t150306-000003-011DA1_ll_gd_rotated.nc
12 =>     /home/arthur/data/cyclobs/rotated_files/clean_dataset/rs2--owi-cm-20200902t084047-20200902t084419-00003-1B372_ll_gd_rotated.nc
13 =>     /home/arthu

ValueError: x and y must have same first dimension, but have shapes (321,) and (5, 321)

In [ ]:
a = np.array([0.8, 0.9, 0.9, 1.1, 1.2, 1.5, 1.7, 1.9, 2.1, 2.3])
b = [np.argwhere(a > i)[0] for i in range(0, 3)]
b

In [ ]:
cat = np.array(ds['current_category'])
print(cat)
test = str(cat)
aa = [i for i in test]
print(aa)

In [ ]:
print(ds)

In [ ]:
r_star = np.linspace(0., 8., num=321)
r_star

In [ ]:
r2 = 1.002355765
st = 'R^2 = {:.2f}'.format(r2)

In [ ]:
a = [2, 2, 2]
b = [1, 1, 1]
np.subtract(a, b)

In [ ]:
def get_std(x, n):
    gap = (x - np.divide(x, n)) ** 2
    std = np.divide(gap, n)
    return std

x = [1, 2, 3]
n = [1, 1, 1]
print(get_std(x, n))
print(np.std(x))

In [17]:
a = np.expand_dims(np.array([1, 1, 1]), axis=0)
b = np.expand_dims(np.array([1, 3, 1]), axis=0)
b2 = np.expand_dims(np.array([1, 4, 1]), axis=0)
c = np.append(a, b, axis=0)
c = np.append(c, b2, axis=0)
print(c)
print(c.shape)

def get_mean_std(x, n):
    masked = np.ma.masked_where(n == 0, x)
    mean   = np.ma.mean(masked, axis=0)
    std    = np.ma.std(masked, axis=0)
    return mean, std

print(get_mean_std(c, np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])))

# mean_diff = np.divide(np.sum(DIFF[i][profile], axis=0), NB_CAT[i][profile])

[[1 1 1]
 [1 3 1]
 [1 4 1]]
(3, 3)
[[1 1 1]
 [1 3 1]
 [1 4 1]]
[1.         2.66666667 1.        ]
MEAN: [1.         2.66666667 1.        ] 
 STD: [0.0 1.247219128924647 0.0]
(3,)
(3,)
(masked_array(data=[1.        , 2.66666667, 1.        ],
             mask=False,
       fill_value=1e+20), masked_array(data=[0.0, 1.247219128924647, 0.0],
             mask=[False, False, False],
       fill_value=1e+20))
